In this project I will work with data from the CIA World Factbook, a compendium of statistics about all countries on Earth. You can download the SQLite factbook.db database by clicking the Download button [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db)

In [5]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [6]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table'

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [8]:
%%sql
SELECT * FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


I'll get started by calculating summary statistics for any outlier countries

In [10]:
%%sql
SELECT MIN(population), MAX(population),
        MIN(population_growth), MAX(population_growth)
    FROM facts

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In [12]:
%%sql
SELECT name FROM facts
WHERE population == 0

Done.


name
Antarctica


In [13]:
%%sql
SELECT name FROM facts
WHERE population == (SELECT MAX(population) FROM facts)

Done.


name
World


Country with the minimum population is Antarctica.
The whole world is considered a country, so that has the highest population. Will recalculate excluding world.

In [25]:
%%sql
SELECT name AS 'Min Population', population FROM facts
WHERE population == (SELECT MIN(population) FROM facts)

Done.


Min Population,population
Antarctica,0


In [24]:
%%sql
SELECT name AS 'Max Population', population FROM facts
WHERE population == (SELECT MAX(population) FROM facts 
                     WHERE name != 'World')

Done.


Max Population,population
China,1367485388


In [26]:
%%sql
select name as 'Min Population Growth', population_growth from facts
where population_growth == (select min(population_growth) from facts)

Done.


Min Population Growth,population_growth
Holy See (Vatican City),0.0
Cocos (Keeling) Islands,0.0
Greenland,0.0
Pitcairn Islands,0.0


In [27]:
%%sql
select name as 'Max Population Growth', population_growth from facts
where population_growth == (select max(population_growth) from facts) 

Done.


Max Population Growth,population_growth
South Sudan,4.02


Antarctica has a population of 0.
China has the largest population at 1.37 Billion
Holy See, Cocos Islands, Greenland, and Pitcairn Islands all had no population growth.
South Sudan had the largest population growth at 4.02%

I will now find the densely populated countries by identifing countries with above average population and below average values for area.

In [33]:
%%sql
SELECT name, CAST(population AS FLOAT) / area AS 'Population Density'
FROM facts
WHERE population > (SELECT AVG(population) FROM facts)
AND area < (SELECT AVG(area) FROM facts)

Done.


name,Population Density
Bangladesh,1138.0691432035565
Germany,226.4689795026637
Japan,335.8418136353413
Philippines,336.6612533333333
Thailand,132.47662340193327
United Kingdom,263.0771396904889
Vietnam,284.8610700160019


These countries have high populations with low areas, but what about high populations with high areas and low populations with low areas that have larger population densities than these?

In [37]:
%%sql
SELECT name, CAST(population AS FLOAT) / area 'Population Density'
FROM facts
ORDER BY CAST(population AS FLOAT) / area DESC
LIMIT 10

Done.


name,Population Density
Macau,21168.964285714286
Monaco,15267.5
Singapore,8141.279770444763
Hong Kong,6445.041516245487
Gaza Strip,5191.819444444444
Gibraltar,4876.333333333333
Bahrain,1771.8592105263158
Maldives,1319.6409395973155
Malta,1310.01582278481
Bermuda,1299.9259259259259


None of the countries with large population and low area are in the top 10 most densely populated.